<img src='./imgs/ssd.png'>

Yolo는 속도 측면에서 비약적인 발전을 이루지만 정확도 측면에선 다소 한계점이 있었습니다. 또한 작은 물체들은 잘 잡아내지 못한다는 문제가 있었습니다. SSD는 바로 이러한 한계점을 극복하고자 하는 시도에서 출발하게 됩니다.

 
### 핵심 아이디어
Yolo v1의 문제점은 입력 이미지를 7x7 크기의 그리드로 나누고, 각 그리드 별로 Bounding Box Prediction을 진행하기 때문에 그리드 크기보다 작은 물체를 잡아내지 못하는 문제가 있었습니다. 그리고 신경망을 모두 통과하면서 컨볼루션과 풀링을 거쳐 coarse한 정보만 남은 마지막 단 피쳐맵만 사용하기 때문에 정확도가 하락하는 한계가 있었습니다. 이에 SSD는 이전 리서치들에서 장점을 모아서 yolo의 한계점을 극복하려 합니다.

"Fully Convolution Network에서 처럼 앞단 컨볼루션 피쳐맵을 끌어와 사용하여 detail을 잡아내고
Faster RCNN의 anchor 개념을 가져와 다양한 형태의 object들도 잡아낼 수 있도록 한다." 


### Multi Scale Feature Maps for Detection

<img src='./imgs/ssd1.png'>

SSD는 Yolo와 달리 컨볼루션 과정을 거치는 중간 중간 피쳐맵들에서 모두 Object Detection을 수행합니다. 

SSD는 먼저 300x300 크기의 이미지를 입력받아서 이미지 넷으로 pretrained된 VGG의 Conv5_3층까지 통과하며 피쳐를 추출합니다. 그 다음 이렇게 추출된 피쳐맵을 컨볼루션을 거쳐 그 다음 층에 넘겨주는 동시에 Object Detection을 수행합니다. 이전 Fully Convolution Network[3]에서 컨볼루션을 거치면서 디테일한 정보들이 사라지는 문제점을 앞단의 피쳐맵들을 끌어오는 방식으로 해결하였습니다. (더 자세한 내용은 이전 포스팅[4]을 참고해주시기 바랍니다.) 

SSD는 여기서 착안하여 각 단계별 피쳐맵에서 모두 Object Detection을 수행하는 방식을 적용한 것입니다. 

<img src='./imgs/ssd2.png'>

VGG를 통과하며 얻은 피쳐맵을 대상으로 쭉쭉 컨볼루션을 진행하여 최종적으로는 1x1 크기의 피쳐맵까지 뽑습니다. 그리고 각 단계별로 추출된 피쳐맵은 Detector & Classifier를 통과시켜 Object Detection을 수행합니다.

<img src='./imgs/ssd3.png'>

컨볼루션 중간에 5x5x256 크기의 피쳐맵을 대상으로 Object Detection을 수행한다고 가정하겠습니다. 여기서 5x5는 Yolo에서 그리드 크기에 해당한다고 생각하시면 됩니다. 이제 하나의 그리드마다 크기가 각기 다른 Default Box들을 먼저 계산합니다. Default Box란 Faster R-CNN에서 anchor의 개념으로 비율과 크기가 각기 다른 기본 박스를 먼저 설정해놓아서 Bounding Box를 추론하는데 도움을 주는 장치이며 그림으로 나타내면 아래와 같습니다.

<img src='./imgs/ssd3.png'>

위 그림을 보면 고양이는 작은 물체이고 강아지는 상대적으로 더 큽니다. 높은 해상도의 피쳐맵에서는 작은 물체를 잘 잡아낼 수 있고, 낮은상도에서는 큰 물체를 잘 잡아낼 것이라고 추측할 수 있습니다. SSD는 각각의 피쳐맵을 가져와서 비율과 크기가 각기 다른 Default Box를 투영합니다. 그리고 이렇게 찾아낸 박스들에 bounding box regression를 적용하고 Confidence Level을 계산합니다. 이는 Yolo가 아무런 기본 값 없이 2개의 Box를 예측하도록 한 것과 대조적입니다.

 

다음으로 피쳐맵에 3x3 컨볼루션을 적용하여(패딩을 1로 설정하여 크기 보존) bounding box regression 값을 계산합니다. 이는 각각의 default box들의 x, y, w, h의 조절 값을 나타내므로 4차원 벡터에 해당하며,  위 그림에서는 인덱스 하나에 3개의 Default Box를 적용하였으므로 결과 피쳐맵의 크기는 5x5x12가 됩니다. 

 

마지막으로 각각의 Default Box마다 모든 클래스에 대하여 클래시피케이션을 진행하는데, 총 20개의 클래스 + 1 (배경 클래스) x Default Box 수이므로 최종 피쳐맵 결과의 크기는 5x5x63이 됩니다.

 

이렇게 각 층별 피쳐 맵들을 가져와 Object Detection을 수행한 결과들을 모두 합하여 로스를 구한 다음, 전체 네트워크를 학습시키는 방식으로 1 Step end-to-end Object Detection 모델을 구성합니다.